Human Computer Interaction CS449 – CS549

Assignment 5: Development of Gesture-based interaction using Mediapipe

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

# MediaPipe için modüller
mp_face_mesh = mp.solutions.face_mesh

# Landmark index'leri
NOSE_TIP = 1
LEFT_EAR = 234
RIGHT_EAR = 454
CHIN = 152

# Kalibrasyon için veriler
default_yaw = 0
default_pitch = 0
yaw_range = {"left": 0, "right": 0}
pitch_range = {"up": 0, "down": 0}

yaw_values = []
pitch_values = []
calibration_step = 0  # 0: varsayılan, 1: sağa, 2: sola, 3: yukarı, 4: aşağı

# Hareket doğrulama
last_movement = None
movement_start_time = None
movement_confirmation_time = 2  # Kullanıcının hareketi doğrulamak için beklemesi gereken süre (saniye)


# Kalibrasyon işlevi
def calibrate_yaw_pitch(yaw, pitch):
    global calibration_step, default_yaw, default_pitch, yaw_range, pitch_range, yaw_values, pitch_values

    if calibration_step == 0 and len(yaw_values) == 0:
        print("Lütfen sabit durun, kalibrasyon başlıyor...")

    if calibration_step == 0:  # Varsayılan pozisyon
        yaw_values.append(yaw)
        pitch_values.append((chin[1] - nose[1]) / (right_ear[0] - left_ear[0]))
        if len(yaw_values) >= 200:  # Daha fazla veri alıyoruz
            default_yaw = np.mean(yaw_values)
            default_pitch = np.mean(pitch_values)  # Default pitch ayarlanır
            print(f"Kalibrasyon tamamlandı: Default Yaw = {default_yaw:.2f}, Default Pitch = {default_pitch:.2f}")
            yaw_values.clear()
            pitch_values.clear()
            calibration_step = 1  # Sağa çevirme

    elif calibration_step == 1:  # Sağa çevirme
        yaw_values.append(yaw)
        if len(yaw_values) >= 100:  # Daha fazla veri alıyoruz
            yaw_range["right"] = min(yaw_values)  # Sağ için min değer
            print(f"Min Sağ Yaw = {yaw_range['right']:.2f}")
            yaw_values.clear()
            calibration_step = 2  # Sola çevirme

    elif calibration_step == 2:  # Sola çevirme
        yaw_values.append(yaw)
        if len(yaw_values) >= 100:
            yaw_range["left"] = max(yaw_values)  # Sol için max değer
            print(f"Max Sol Yaw = {yaw_range['left']:.2f}")
            yaw_values.clear()
            calibration_step = 3  # Yukarı bakma

    elif calibration_step == 3:  # Yukarı bakma
        pitch_values.append(pitch)
        if len(pitch_values) >= 100:
            pitch_range["up"] = max(pitch_values)  # Yukarı için min değer
            print(f"Max Yukarı Pitch = {pitch_range['up']:.2f}")
            pitch_values.clear()
            calibration_step = 4  # Aşağı bakma

    elif calibration_step == 4:  # Aşağı bakma
        pitch_values.append(pitch)
        if len(pitch_values) >= 100:
            pitch_range["down"] = min(pitch_values)  # Aşağı için max değer
            print(f"Min Aşağı Pitch = {pitch_range['down']:.2f}")
            print("Kalibrasyon tamamlandı!")
            calibration_step = -1  # Kalibrasyon tamamlandı


def detect_head_movement(yaw, pitch):
    global default_yaw, default_pitch, yaw_range, pitch_range, last_movement, movement_start_time

    current_movement = None

    # Yaw hareketi algılama
    if yaw <= yaw_range["right"] / 2:  # Sağ için min değer
        current_movement = "Baş sağa çevrildi!"
    elif yaw >= yaw_range["left"] / 2:  # Sol için max değer
        current_movement = "Baş sola çevrildi!"

    # Pitch hareketi algılama
    if pitch <= pitch_range["down"] / 2:  # Aşağı için min değer
        current_movement = "Baş aşağı indirildi!"
    elif pitch >= pitch_range["up"] / 2:  # Yukarı için max değer
        current_movement = "Baş yukarı kaldırıldı!"

    # Hareketi doğrulama
    if current_movement == last_movement:
        if movement_start_time is None:
            movement_start_time = time.time()
        elif time.time() - movement_start_time >= movement_confirmation_time:
            if current_movement:  # Sadece bir hareket varsa yazdır
                print(current_movement)
            movement_start_time = None  # Doğrulandıktan sonra sıfırla
    else:
        last_movement = current_movement
        movement_start_time = None  # Yeni hareket için zamanlayıcıyı sıfırla


# Kamerayı aç
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Kamera bulunamadı!")
            break

        # Renk uzayını değiştir
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Landmark'ların koordinatlarını al
                landmarks = face_landmarks.landmark
                nose = np.array([landmarks[NOSE_TIP].x, landmarks[NOSE_TIP].y])
                left_ear = np.array([landmarks[LEFT_EAR].x, landmarks[LEFT_EAR].y])
                right_ear = np.array([landmarks[RIGHT_EAR].x, landmarks[RIGHT_EAR].y])
                chin = np.array([landmarks[CHIN].x, landmarks[CHIN].y])

                # Yaw ve Pitch hesaplama
                yaw = nose[0] - ((left_ear[0] + right_ear[0]) / 2)  # Daha hassas yaw hesaplama

                # Yaw ve Pitch hesaplama
                yaw = nose[0] - ((left_ear[0] + right_ear[0]) / 2)  # Daha hassas yaw hesaplama

                # Daha hassas pitch hesaplama ve normalize etme
                pitch = ((chin[1] - nose[1]) / (right_ear[0] - left_ear[0])) - default_pitch  # Default pitch değerini sıfırlama

                # Kalibrasyonu yönet
                calibrate_yaw_pitch(yaw, pitch)

                # Kalibrasyon sonrası hareket algılama
                if calibration_step == -1:
                    detect_head_movement(yaw, pitch)

                # Bilgileri ekrana yazdır
                cv2.putText(frame, f"Yaw: {yaw:.2f}, Pitch: {pitch:.2f}",
                            (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 4)

                # Kullanıcıya görsel mesajlar
                if calibration_step == 1:
                    cv2.putText(frame, "Lutfen basinizi saga cevirin.",
                                (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4)
                elif calibration_step == 2:
                    cv2.putText(frame, "Lutfen basinizi sola cevirin.",
                                (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4)
                elif calibration_step == 3:
                    cv2.putText(frame, "Lutfen basinizi yukari kaldirin.",
                                (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4)
                elif calibration_step == 4:
                    cv2.putText(frame, "Lutfen basinizi asagi indirin.",
                                (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4)
                elif calibration_step == -1:
                    cv2.putText(frame, "Kalibrasyon tamamlandı!",
                                (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4)

        # Kamerayı göster
        cv2.imshow("Bas Hareketleri Algilama", frame)

        # Çıkmak için 'q' tuşuna basın
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1733682915.681392 2649799 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733682915.692675 2649945 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733682915.695390 2649942 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733682915.708544 2649944 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Lütfen sabit durun, kalibrasyon başlıyor...


2024-12-08 21:35:16.121 Python[53742:2649799] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-08 21:35:16.121 Python[53742:2649799] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Kalibrasyon tamamlandı: Default Yaw = 0.01, Default Pitch = 1.09
Min Sağ Yaw = -0.08
Max Sol Yaw = 0.09
Max Yukarı Pitch = 0.27
Min Aşağı Pitch = -0.32
Kalibrasyon tamamlandı!
Baş sağa çevrildi!
Baş sağa çevrildi!
Baş sola çevrildi!
Baş yukarı kaldırıldı!
Baş aşağı indirildi!


: 